<a href="https://colab.research.google.com/github/prithwis/parashar21/blob/main/P21_45_08_TransitTracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](http://1.bp.blogspot.com/_5QlanosqxiQ/SXLOLTu7S7I/AAAAAAAAAm8/0r6C7lCxsic/S1600-R/p21-header-003.png) <br>


---



[Prithwis Mukerjee](http://www.yantrajaal.com) ||
Website - [Parashar21](https://parashar21.blogspot.com)<br>

Copyright (c) 2022, Prithwis Mukerjee All rights reserved.

This source code is licensed under the GNU GPL v3.0 -style license found in the LICENSE file in the root directory of this source tree.

#Rationale
Use Swiss Ephemeris software to track transits. <br>
Extra fun in learning ffmpeg and making a movie out of this



#SetUp Environment

##Install Software

In [1]:
!pip -qq install pyswisseph                                 # https://stackoverflow.com/questions/64277506/pip-install-options-unclear
!pip -qq install python-docx                                #https://python-docx.readthedocs.io/en/latest/
!pip -qq install pymongo
#!/usr/bin/python3 -m pip install "pymongo[srv]"            # Required only if used with Atlas remote servers
#!apt install mongodb > /dev/null                            # install local MongoDB server
#!service mongodb start                                      # start Local MongoDB server instance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.6/603.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 24.8 MB/s eta 0:00:00


##Install, import prerequisites

In [2]:
#Utility functions
from pymongo import MongoClient
import pandas as pd
import dateutil
import json
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from datetime import datetime
from datetime import timedelta
import pytz
from google.colab import files

#import matplotlib.pyplot as plt
#import math

import os
from IPython.display import HTML
from base64 import b64encode

In [3]:
#Load p21 modules from github
!wget -O p21.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21.py                  # all global variables
!wget -O p21utils.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utils.py        # large number of utility functions
!wget -O p21utilsR.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utilsR.py      # report writing utility functions
!wget -O p21swe.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21swe.py            # Swiss Ephemeris functions
!wget -O p21YogInfo.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21YogInfo.py    # yog data - description and conditions
# ---------------------------

#!wget -O Saraswati.png -q https://github.com/prithwis/parashar21/raw/main/images/Saraswati02.png
#!wget -O p21logo.png -q https://github.com/prithwis/parashar21/raw/main/images/p21logo-002-1.png

#!gdown -O ClockTicking01.mp3 1R6QQ3hTMlZh_k4x_pS0Masfj6SJZ2bgH
!wget -O ClockTicking01.mp3 -q https://github.com/prithwis/parashar21/blob/main/images/ClockTicking01.mp3?raw=true
!wget -O p21Logo.png -q https://github.com/prithwis/parashar21/blob/main/images/p21Logo.png?raw=true


In [4]:
import p21
import p21utils
import p21utilsR
import p21swe
#import p21YogInfo
# ------------------------------------------------
# required for multiple imports of the same module
# ------------------------------------------------
import importlib
importlib.reload(p21)
importlib.reload(p21utils)
importlib.reload(p21utilsR)
importlib.reload(p21swe)
#importlib.reload(p21YogInfo)

10 Yogs on record
0 : Amala Yog | 10th from Moon OR Lagna should have a benefic
1 : RajLakshana Yog | Jupiter, Venus, Mercury, Moon should be in Lagna or they should be in a Kendra
2 : Chatussagara Yog | All the Kendras 1/4/7/10 are occupied by planets, Ra, Ke not counted here
3 : Adhi678 Yog | Benefics located in 6/7/8th  from Moon
4 : Chandra Mangal Yog | Mars Conjoins Moon or  Mars and Moon in Mutual Aspect
5 : Kemadruma Yog : No planets on either sides of the Moon
6 : Anapha Yog |Planets other than Sun in 12th Place from Moon
7 : Sunapha Yog |Planets other than Sun in 2nd Place from Moon
8 : Dhurdhura Yog | Planets on both sides of the Moon
9 : Gajakesari Yog | Jupiter is in Kendra or 1,4,7,10th from Moon


<module 'p21swe' from '/content/p21swe.py'>

In [5]:
#Download Swiss Ephemeris Files
!wget -q http://www.astro.com/ftp/swisseph/ephe/seas_18.se1
!wget -q http://www.astro.com/ftp/swisseph/ephe/semo_18.se1
!wget -q http://www.astro.com/ftp/swisseph/ephe/sepl_18.se1
!mkdir ephe
!mv *.se1 ephe
#------------------------------
# Configure SWE with appropriate parameters
#
p21swe.C01_configSWE()

In [6]:
#@title Static Data
# @markdown ### Specify Reference Rashi:
rRashi = 'Dhanu' #@param ["Mesh","Vrish","Mithun","Karkat","Simha","Kanya","Tula","Vrishchik","Dhanu","Makar","Kumbh","Meen"]
#@markdown ---
#@markdown ### Using Calcutta as default location:
PoB_Lat = 25.43 #@param {type:"number"}
PoB_Lon = 81.85 #@param {type:"number"}
TZ_OffHours = 5.5
DoB_Time = '12:01'
#PoB_Lat = 25.43
#PoB_Lon = 81.85
Name = p21.gName
Gender = 'x'
tag1 = 'nil'
tag3 = 'nil'
tag5 = 'nil'
p21.pName = p21.gName
p21.ChartType = 'Rashi'
RashiNumber = p21.RashiName.index(rRashi)
RashiLon = RashiNumber * 30 - 15
p21.SubMoonLong = RashiLon    # Needed in Gochar 'La' long

In [7]:
print(RashiNumber, RashiLon)
#p21.SubMoonLong = RashiLon
print(p21.SubMoonLong)

9 255
255


In [8]:
#current = datetime.now(pytz.timezone('Asia/Kolkata'))
#print(current)

In [9]:
def getDelta(cD,bA,aA,bR,aR,AllTransits = False):
    #print('getDelta',cD)
    cDs = cD.strftime("%d %b %y")+'|'
    grahaList = p21.Graha + ['Ra','Ke']
    #for G in p21.Graha:
    for G in grahaList:
        if bA[G] != aA[G]:
            if G not in ['Mo', 'Me']:               # Excludes certain grahas to reduce the clutter
                print(cDs, G, 'TRANSITS from ', bA[G], 'to ', aA[G])
            else:
                if AllTransits:
                    print(cDs, G, 'transits from ', bA[G], 'to ', aA[G])
            #print(cDs, G, 'transits from ', bA[G], 'to ', aA[G])
        if bR['GRet'][G] != aR['GRet'][G]:
            #print(cD,G, 'changes from ', bR['GRet'][G], 'to ', aR['GRet'][G], ' retrograde')
            if bR['GRet'][G]:
                print(cDs, G, 'turns DIRECT')
            else:
                print(cDs, G, 'turns RETRO')

In [10]:
def Transitz(numberOfDays,allTransits = False,buildMovie = False):
    sDate = datetime.now(pytz.timezone('Asia/Kolkata'))
    cDate = sDate

    p21utils.setTransitData(p21utils.getTransitData(cDate))
    for count in range(numberOfDays):

        prevGrashiA = p21.GRashiA
        prevGLonRet = p21.GLonRet
        cDate += timedelta(days=1)

        p21utils.setTransitData(p21utils.getTransitData(cDate))
        if buildMovie:
            p21.cID = cDate.strftime("%d %b %y")
            #p21utilsR.R12B_drawChart_South()
            p21utilsR.R12B_drawChart_Bengal()
            newName = './cImages/img'+str(count)+'.png'
            oldName = './RashiChart.png'
            os.rename(oldName, newName)
        getDelta(cDate,prevGrashiA,p21.GRashiA,prevGLonRet,p21.GLonRet,allTransits)

## Transits as Text


In [11]:
!mkdir ./cImages
!rm ./cImages/*.png

Transitz(150)       # AllTransits = False, BuildMovie = False, buy default

rm: cannot remove './cImages/*.png': No such file or directory
18 Aug 23| Su TRANSITS from  Karkat to  Simha
19 Aug 23| Ma TRANSITS from  Simha to  Kanya
24 Aug 23| Me turns RETRO
04 Sep 23| Ve turns DIRECT
05 Sep 23| Ju turns RETRO
16 Sep 23| Me turns DIRECT
18 Sep 23| Su TRANSITS from  Simha to  Kanya
02 Oct 23| Ve TRANSITS from  Karkat to  Simha
04 Oct 23| Ma TRANSITS from  Kanya to  Tula
18 Oct 23| Su TRANSITS from  Kanya to  Tula
03 Nov 23| Ve TRANSITS from  Simha to  Kanya
05 Nov 23| Sa turns DIRECT
16 Nov 23| Ma TRANSITS from  Tula to  Vrishchik
17 Nov 23| Su TRANSITS from  Tula to  Vrishchik
29 Nov 23| Ra TRANSITS from  Mesh to  Meen
29 Nov 23| Ke TRANSITS from  Tula to  Kanya
30 Nov 23| Ve TRANSITS from  Kanya to  Tula
14 Dec 23| Me turns RETRO
17 Dec 23| Su TRANSITS from  Vrishchik to  Dhanu
25 Dec 23| Ve TRANSITS from  Tula to  Vrishchik
28 Dec 23| Ma TRANSITS from  Vrishchik to  Dhanu
31 Dec 23| Ju turns DIRECT
02 Jan 24| Me turns DIRECT


##Transits as a video
reference : https://ostechnix.com/20-ffmpeg-commands-beginners/

In [12]:
def show_video(video_path, video_width = 600):

  video_file = open(video_path, "r+b").read()

  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

In [13]:
#!mkdir ./cImages
!rm ./cImages/*.png

rm: cannot remove './cImages/*.png': No such file or directory


In [14]:
%%capture
#
# Create Images for the Movie
#
Transitz(200,False,True)

In [15]:
# Create movie
# changing the FPS to 5
#!ffmpeg -f image2 -r 5 -i ./cImages/img%d.png -vcodec mpeg4 -y ./p21Transitz.mp4  # this codec does not work well
#https://shotstack.io/learn/use-ffmpeg-to-convert-images-to-video/

!ffmpeg -loglevel error -framerate 5 -i ./cImages/img%d.png -c:v libx264 -r 5 p21Transitz_0.mp4   # -loglevel error supresses non-error messages


In [16]:
# see the movie
#
show_video('p21Transitz_0.mp4',300)

In [17]:
#Add audio
# https://ostechnix.com/20-ffmpeg-commands-beginners/
#

!ffmpeg -loglevel error -i p21Transitz_0.mp4 -i ClockTicking01.mp3 -shortest -c:v copy -c:a aac p21Transitz_1.mp4

In [18]:
# see the movie
#
show_video('p21Transitz_1.mp4',300)

In [19]:
#Add a branding image
# https://gist.github.com/bennylope/d5d6029fb63648582fed2367ae23cfd6
#
!ffmpeg -loglevel error -i p21Transitz_1.mp4 -i p21Logo.png -filter_complex 'overlay=x=((main_w-overlay_w)/2)-1:y=((main_h-overlay_h)/2)+5' p21Transitz_2.mp4

[libx264 @ 0x5a00124c7340] height not divisible by 2 (562x575)
Error initializing output stream 0:0 -- Error while opening encoder for output stream #0:0 - maybe incorrect parameters such as bit_rate, rate, width or height


In [20]:
# see the movie
#
show_video('p21Transitz_2.mp4',300)

In [21]:
# Convert to Whatsapp format
# https://ostechnix.com/convert-videos-to-whatsapp-video-format-with-ffmpeg/
#

!ffmpeg -loglevel error -i p21Transitz_2.mp4 -vcodec libx264 -acodec aac p21Transitz_w.mp4

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x5c22976fe0c0] moov atom not found
p21Transitz_2.mp4: Invalid data found when processing input


In [22]:
# see the movie
#
show_video('p21Transitz_w.mp4',300)

FileNotFoundError: ignored

In [ ]:
!ffprobe p21Transitz.mp4

#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself.
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)